In [0]:
import numpy as np
from sympy.integrals.quadrature import gauss_lobatto

!pip install numdifftools
import numdifftools as nd

In [0]:
import timeit

# Approssimazione polinomiale dell'inversa

La funzione **polynomial_inversion** prende in input un array $X$, generato con *dataset_full*, il relativo valore della dimensione $d$ ed il valore *up* che indica di quanto aumentare il parametro $n$ dell'inversa polinomiale di $\mathcal{L}_K^n$.

Ci restituisce in output un array contenente un'approssimazione delle $d$ coordinate della preimmagine di $x$, ovvero le coordinate di $\hat{x}: \mathcal{L}_K(\hat{x}) = x$. 

Settando il parametro *eps* (default $10^{-8}$) si può impostare la precisione richiesta al metodo.

In [0]:
def polynomial_inversion(X, d, up=1, eps=1.e-8):
    
    ### t0 = timeit.default_timer()

    p = X[0:d].reshape((-1,d))
    K1 = X[d:].reshape(-1,d)
    n1 = int(pow(K1.shape[0],1/d)+.5)
    n2 = n1 + up

    ##################################################################

    def interpolation_points(n):
        q, w = gauss_lobatto(n, 10)
        q = np.array(q, dtype=float)
        if(d==2):
           Q1, Q2 = np.meshgrid(q,q)
           return np.stack([Q1,Q2], axis=-1).reshape((-1,d))
        elif(d==3):
           Q1, Q2, Q3 = np.meshgrid(q,q,q)
           return np.stack([Q1,Q2,Q3], axis=-1).reshape((-1,d))

    ##################################################################

    Khat1 = interpolation_points(n1)
    Khat2 = interpolation_points(n2).flatten()
    

    ##################################################################

    def Lagrange(i, x, n, Khat):
        x = x.repeat(n-1).reshape((-1,d*(n-1)))
        nodes = Khat[0:n,0]
        v = []
        for k in range(d):
            v.append(np.delete(nodes,np.where(nodes==Khat[i][k])))
        v = np.array(v).flatten()
        v = v.repeat(x.shape[0]).reshape((-1,len(v)), order='F')
        avoid = Khat[i].repeat(n-1).repeat(x.shape[0]).reshape(-1,d*(n-1), order='F')
        return np.prod((x-v)/(avoid-v), axis=1)

    def L(x, K, n, Khat):
        x = np.array(x)
        lgr = np.zeros((0,x.shape[0]))
        for i in range(len(Khat)):
            lgr = np.append(lgr, Lagrange(i,x,n,Khat))
        lgr = np.stack(lgr.reshape((-1,x.shape[0])),axis=-1)
        return np.matmul(lgr,K)

    ###################################################################

    def residual(newCell):
        return L(newCell.reshape((-1,d)), K1, n1, Khat1).flatten() - Khat2

    ##################################################################
    
    ### t1 = timeit.default_timer()
    
    K2 = Khat2
    while(np.linalg.norm(residual(K2))>eps):
          K2 = K2 - np.matmul(np.linalg.inv(nd.Jacobian(residual)(K2)), residual(K2).T)
    K2 = K2.reshape((-1,d))
    
    ####################################################################
    
    ###t2 = timeit.default_timer()

    return L(p,K2,n2,Khat2.reshape((-1,d))).flatten()###, t1-t0, t2-t1, timeit.default_timer()-t2